In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as spio
import getpass
import os
import csv

In [188]:
# Number of days in total
numdays = 6

# Name of the computer
myhost = getpass.getuser()

# Base folder
datafolder = 'C:/Users/username/Documents/Fieldwork/fieldwork-2019-adcp-adv/data/'.replace('username', myhost)# basefolder
basefolder = datafolder+'StreamPro'

# Subfolders
daynames = [i for i in os.listdir(basefolder)]






In [245]:
# Count the number of days
day = 0

# which files to include
includes =['*streampro_datatotal_ASC.TXT']

# loop over all roots
for root, dirs, files in os.walk(basefolder, topdown=True):
  
    # Don't look in the basefolder
    if root == basefolder:
        continue
        
    # which folder are we doing right now?
    print(daynames[day])
    
    goodfiles =[file for file in files if file[-13:] == 'total_ASC.TXT']
    
 
    
    # Loop over the correct files in the folder
    for file in goodfiles:
        # Load direction from file in folder
        print(file)
        data = pd.read_csv(root+'/'+file, delimiter = ';')
        water_vel = pd.DataFrame.as_matrix(data['northV']) #only the velocity to the north is in direction of the river
        print(water_vel[0].split(','))

    day +=1

Monday afternoon
group 5_0_000_streampro_datatotal_ASC.TXT
['0.010', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768']
group 5_0_001_streampro_datatotal_ASC.TXT
['-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768']
group 5_0_002_streampro_datatotal_ASC.TXT
['-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-327

C:\Users\Sanne de Smet\Documents\Python\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


['-0.167', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768']
Station_0_005_streampro_datatotal_ASC.TXT
['-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768']
Station_0_006_streampro_datatotal_ASC.TXT
['0.072', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768', '-32768']
Station_0_007_